## Step 0: Install docker and place sudo password in ~/.sudopassword

### Removed previous run containers (Skip when starting for first time!!)

In [10]:
!sudo -S docker stop /zookeeper-server < ~/.sudopassword
!sudo -S docker stop /kafka-server < ~/.sudopassword
!sudo -S docker stop /event-stream < ~/.sudopassword

!sudo -S docker rm /zookeeper-server < ~/.sudopassword
!sudo -S docker rm /kafka-server < ~/.sudopassword
!sudo -S docker rm /event-stream < ~/.sudopassword
!sudo -S docker network rm app-tier < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream
[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream
[sudo] password for kalinga: app-tier


## Step 1: Create a network

In [11]:
!sudo -S docker network create app-tier --driver bridge < ~/.sudopassword

[sudo] password for kalinga: 88104f97889b24ab12d89ad927dac4eed70cfe12559f43ab8a00616ec9236fd7


## Step 2: Launch the Zookeeper server instance

In [12]:
!sudo -S  docker run -d --name zookeeper-server --network app-tier -e ALLOW_ANONYMOUS_LOGIN=yes bitnami/zookeeper:latest < ~/.sudopassword

[sudo] password for kalinga: 1d2359e7d915277109ff592fea5f7ef9d9a6df385e419474257e0356f6577a45


## Step 3: Launch the Kafka server instance

In [13]:
!sudo -S docker run -d --name kafka-server \
    --network app-tier \
    -e ALLOW_PLAINTEXT_LISTENER=yes \
    -e KAFKA_CFG_ZOOKEEPER_CONNECT=zookeeper-server:2181 \
    bitnami/kafka:latest < ~/.sudopassword

[sudo] password for kalinga: 3f93b65e593edd9b77d7b80fa50815df05b1f609f05edea0b381f309a0113ea0


## Step 4: Launch event-stream  instance

inside the extracted folder  


In [15]:
!sudo -S docker build -t kalinga/event-stream:v0.1 . < ~/.sudopassword

[sudo] password for kalinga: Sending build context to Docker daemon  62.98kB
Step 1/8 : FROM python:3
 ---> b162b5322f1c
Step 2/8 : LABEL Author="Kalinga Bhusan Ray"
 ---> Using cache
 ---> 55e95b0d90db
Step 3/8 : LABEL E-Mail="kalinga-bhusan.ray@tu-ilmenau.de"
 ---> Using cache
 ---> 5e490150ac8b
Step 4/8 : LABEL Desc="This is an image for event stream engiene"
 ---> Using cache
 ---> 430a9dc7ecdd
Step 5/8 : RUN apt-get update &&     apt-get install -y vim
 ---> Using cache
 ---> 63f97c80a8c5
Step 6/8 : ADD process_event_stream.py /
 ---> 2b10727ef7cf
Step 7/8 : RUN pip install faust typing_extensions
 ---> Running in 795a2805d164
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp38-none-any.whl size=15355 sha256=e7b8ab5c24701d1e7273abde06d6e94e9687cbb307c5c899ef80750b91b405e3
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
  Created wheel for opentracing: filename=opentracing-1.3.0-cp38-none-any.whl size=184

In [16]:
!sudo -S docker run -d \
                -it  \
                --name event-stream \
                --mount type=bind,source="$(pwd)"/.,target=/app \
                --network app-tier \
                kalinga/event-stream:v0.1 /bin/bash < ~/.sudopassword    

[sudo] password for kalinga: 58ee497bf42e310da80fecfc368bde02d88e0b0a91b2abcf6fe6591c5e477fe2


In [17]:
!sudo -S docker ps  < ~/.sudopassword

[sudo] password for kalinga: CONTAINER ID        IMAGE                       COMMAND                  CREATED             STATUS                  PORTS                                    NAMES
58ee497bf42e        kalinga/event-stream:v0.1   "/bin/bash"              5 seconds ago       Up Less than a second   80/tcp                                   event-stream
3f93b65e593e        bitnami/kafka:latest        "/entrypoint.sh /run…"   2 minutes ago       Up 2 minutes            9092/tcp                                 kafka-server
1d2359e7d915        bitnami/zookeeper:latest    "/entrypoint.sh /run…"   2 minutes ago       Up 2 minutes            2181/tcp, 2888/tcp, 3888/tcp, 8080/tcp   zookeeper-server


In [18]:
!ls

containerFS	    operations		     README.md		    ut-acc-init
Dockerfile	    process_event_stream.py  stream-event-app-data  ut-order
event-stream.ipynb  __pycache__		     test.py


### For existing containers (During development after restarting the host machine)

In [1]:
!sudo -S docker start /zookeeper-server < ~/.sudopassword
!sudo -S docker start /kafka-server < ~/.sudopassword
!sudo -S docker start /event-stream < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream


In [2]:
!sudo -S docker ps < ~/.sudopassword

[sudo] password for kalinga: CONTAINER ID        IMAGE                       COMMAND                  CREATED             STATUS              PORTS                                    NAMES
629d5f35935b        kalinga/event-stream:v0.1   "/bin/bash"              2 days ago          Up 10 seconds       80/tcp                                   event-stream
315d2f8ea9fc        bitnami/kafka:latest        "/entrypoint.sh /run…"   2 days ago          Up 18 seconds       9092/tcp                                 kafka-server
ae3034974523        bitnami/zookeeper:latest    "/entrypoint.sh /run…"   2 days ago          Up 25 seconds       2181/tcp, 2888/tcp, 3888/tcp, 8080/tcp   zookeeper-server


### Step 5: Copy the Unit tests

In [23]:
!sudo -S docker cp  ut-acc-init kafka-server:/ < ~/.sudopassword
!sudo -S docker cp  ut-order kafka-server:/ < ~/.sudopassword
!sudo -S docker cp  ut-order-doubled kafka-server:/ < ~/.sudopassword
!sudo -S docker cp  ut-order-high-frequence kafka-server:/ < ~/.sudopassword    
!sudo -S docker cp  ut-insufficient kafka-server:/ < ~/.sudopassword        

[sudo] password for kalinga: [sudo] password for kalinga: [sudo] password for kalinga: 

### Step 6: Start the app and test

#### [In separate terminals]
docker exec -it event-stream bash  
cd app  
faust -A process_event_stream worker -l info  

docker exec -it kafka-server bash  

[kafka-server terminal]  
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic transaction --producer.config /opt/bitnami/kafka/conf/producer.properties <  ut-acc-init  
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic transaction --producer.config /opt/bitnami/kafka/conf/producer.properties <  ut-order   

### Step 6: Extend the requirements

Not yet implemented:  
    `high-frequency-small-interval`  
    `doubled-transaction`


#### Development shortcuts

In [7]:
!sudo -S docker run -d \
                -it  \
                --name event-stream \
                --mount type=bind,source="$(pwd)"/.,target=/app \
                --network app-tier \
                kalinga/event-stream:v0.1 /bin/bash < ~/.sudopassword    

[sudo] password for kalinga: db971051eb54966f51e53d7a1759410614a1f8cc75983b362ee06768ae146a34


In [4]:
#!sudo -S docker container ls --all < ~/.sudopassword

In [ ]:
!sudo -S docker cp event-stream:/process_event_stream.py .  < ~/.sudopassword

In [ ]:
!sudo -S chown kalinga:kalinga process_event_stream.py < ~/.sudopassword

In [2]:
!sudo -S docker stop event-stream < ~/.sudopassword

[sudo] password for kalinga: event-stream


In [6]:
!sudo -S docker rm /event-stream < ~/.sudopassword

[sudo] password for kalinga: /event-stream


In [20]:
!sudo -S docker stop /zookeeper-server < ~/.sudopassword
!sudo -S docker stop /kafka-server < ~/.sudopassword
!sudo -S docker stop /event-stream < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream


In [25]:
#kafka-topics.sh --zookeeper zookeeper-server:2181 --list

In [24]:
#kafka-topics.sh --zookeeper zookeeper-server:2181 --delete --topic stream-event-app-account-changelog

## Fresh start the kafka

In [22]:
!sudo -S docker stop /zookeeper-server < ~/.sudopassword
!sudo -S docker stop /kafka-server < ~/.sudopassword

!sudo -S docker rm /zookeeper-server < ~/.sudopassword
!sudo -S docker rm /kafka-server < ~/.sudopassword

!sudo -S  docker run -d --name zookeeper-server --network app-tier -e ALLOW_ANONYMOUS_LOGIN=yes bitnami/zookeeper:latest < ~/.sudopassword
!sudo -S docker run -d --name kafka-server \
    --network app-tier \
    -e ALLOW_PLAINTEXT_LISTENER=yes \
    -e KAFKA_CFG_ZOOKEEPER_CONNECT=zookeeper-server:2181 \
    bitnami/kafka:latest < ~/.sudopassword

!sudo -S docker cp  ut-acc-init kafka-server:/ < ~/.sudopassword
!sudo -S docker cp  ut-order kafka-server:/ < ~/.sudopassword
!sudo -S docker cp  ut-order-doubled kafka-server:/ < ~/.sudopassword
!sudo -S docker cp  ut-insufficient kafka-server:/ < ~/.sudopassword        

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: 8430f3d7a232af05461dfeed57f0c965e51ce31c9df3d4074c5bbc4267716891
[sudo] password for kalinga: 6d672f311ed1e732e0bf577c2662e8efb9aff0d5ff5b8d7d675c35599e4a7cd9


In [6]:
!sudo -S docker cp  ut-insufficient kafka-server:/ < ~/.sudopassword    

[sudo] password for kalinga: 